# PRO3515 - PMP LPT/SPT


## Dupla 9

### Nomes: 
### Lucas Hideki Takeuchi Okamura NUSP:  9274315
### Thales Arantes Kerche Nunes   NUSP: 10769372


### Objetivo
        * Minimizar o tempo total (makespan) na programação de máquinas paralelas (PMP - Parallel Machine Problem),
        utilizando a regra do maior tempo de processamento (LPT - longest processing time)

Importando a biblioteca sugerida

In [1]:
import xlwings as xw

Definindo as classes de Job e Maq que usaremos no algoritmo

In [2]:
class Job:
    def __init__(self,i,pi,di):
        self.i=i     # número do job, pela ordem de chegada
        self.p=pi     # processing time
        self.d=di     # due date
        self.C=0     # completion time
        self.T=0     # tardiness

In [3]:
class Maq:
    def __init__(self,k):
        self.k=k     # índice da máquina
        self.jobs=[]     # jobs alocados à máquina
        self.carga=0.0     # carga alocada à máquina

### Reutilizamos a função disponibilizada pelo professor, com alguns ajustes para ler o número de máquinas

In [4]:
def leInst(sheet_num):
    pl1 = wb1.sheets[sheet_num]
    m=int(pl1.range('B5').value)     # converter em inteiro
    n=int(pl1.range('B6').value)     # converter em inteiro
    tab1=pl1.range('A11').options(expand='table', numbers=int).value     # ler como tabela e inteiros
    print()
    print('n =',n)
    print('m =',m)
    print('dados =',tab1)
    jobs=[]
    for i in range(n):
        pi=tab1[i][1]
        jobs.append(Job(i,pi,0))
    
    return m,n,jobs

Também reutilizamos a de gravar, novamente, com alguns ajustes

In [5]:
def gravaSched(maqs,m,regra):

    n=len(jobs)
    tab1=[]
    print(regra)
    print('{:2s} {:2s} {:2s} {:2s}'.format('maq','#','p','C'))
    for maq in maqs:
        for job in maq.jobs:
            tab1.append([maq.k,job.i, job.p, job.C])
            print('{:2d} {:2d} {:2d} {:2d}'.format(maq.k, job.i, job.p, job.C))
    
    i=wb2.sheets.count
    pl1=wb2.sheets.add(regra,after=i)
    pl1.range('A5').value=['m',m]
    pl1.range('A6').value=['n',n]
    pl1.range('E5').value=['Fbar',Fbar]
    pl1.range('E6').value=['Cmax',Cmax]
    pl1.range('A10').value=['maq','#','p','C']
    pl1.range('A11').value=tab1

    print()
    print('Cmax: {:4.1f}'.format(Cmax))
    print('Fbar: {:4.1f}'.format(Fbar))

### Procedimento
    1. Ordenar os jobs em ordem decrescente dos tempos de processamento (LPT).

    2. Seguindo a ordem LPT, alocar os jobs na máquina com menor carga (soma dos tempos de processamento dos jobs já alocados). Em caso de empate, escolher arbitrariamente uma entre as máquinas com menor carga (p.ex: máquina de menor índice).

    3. Definidas as alocações, ordenar os jobs em cada máquina em ordem crescente dos tempos (SPT).
    
    4. Calcular os indicadores Cmax e Fbar.

In [6]:
def calcInd(jobs, m, n):     # Indicadores para SMP
    
    maqs = [Maq(k+1) for k in range(m)]
    jobs.sort(key=lambda job: job.p, reverse=True)
    for job in jobs:
        minim = 9999999999
        for ind in range(len(maqs)):
            if maqs[ind].carga < minim:
                minim = maqs[ind].carga
                achou = ind
        maqs[achou].jobs.append(job)
        maqs[achou].carga += job.p

    Cmax=0
    for maq in maqs:
        maq.jobs.sort(key=lambda job: job.p)
        maq.jobs[0].C=maq.jobs[0].p
        for j in range(1,len(maq.jobs)):
            maq.jobs[j].C=maq.jobs[j-1].C+maq.jobs[j].p
        if maq.carga > Cmax:
            Cmax = maq.carga
    
    Fbar = sum([job.C for maq in maqs for job in maq.jobs])/n

    return maqs, Cmax, Fbar

## Principal

Abre os dois arquivos que vão ser usados

In [7]:
wb1 = xw.Book("xl06 3 A PMP Cmax LPT-SPT.xlsx")
wb2 = xw.Book()

Teste 1:

In [8]:
m,n,jobs=leInst(0)

maqs, Cmax, Fbar = calcInd(jobs, m, n)

gravaSched(maqs, m, 'I(1)')


n = 5
m = 2
dados = [[1, 3], [2, 4], [3, 2], [4, 6], [5, 1]]
I(1)
maq #  p  C 
 1  2  2  2
 1  3  6  8
 2  4  1  1
 2  0  3  4
 2  1  4  8

Cmax:  8.0
Fbar:  4.6


Teste 2:

In [9]:
m,n,jobs=leInst(1)

maqs, Cmax, Fbar = calcInd(jobs, m, n)

gravaSched(maqs, m, 'I(2)')


n = 10
m = 3
dados = [[1, 4], [2, 2], [3, 9], [4, 12], [5, 15], [6, 1], [7, 5], [8, 1], [9, 3], [10, 7]]
I(2)
maq #  p  C 
 1  5  1  1
 1  0  4  5
 1  4 15 20
 2  7  1  1
 2  1  2  3
 2  6  5  8
 2  3 12 20
 3  8  3  3
 3  9  7 10
 3  2  9 19

Cmax: 20.0
Fbar:  9.0


Teste 3:

In [10]:
m,n,jobs=leInst(2)

maqs, Cmax, Fbar = calcInd(jobs, m, n)

gravaSched(maqs, m, 'I(3)')


n = 12
m = 3
dados = [[1, 12], [2, 6], [3, 7], [4, 8], [5, 2], [6, 3], [7, 15], [8, 16], [9, 20], [10, 13], [11, 17], [12, 10]]
I(3)
maq #  p  C 
 1  1  6  6
 1  2  7 13
 1  0 12 25
 1  8 20 45
 2  4  2  2
 2 11 10 12
 2  9 13 25
 2 10 17 42
 3  5  3  3
 3  3  8 11
 3  6 15 26
 3  7 16 42

Cmax: 45.0
Fbar: 21.0


Teste 4:

In [11]:
m,n,jobs=leInst(3)

maqs, Cmax, Fbar = calcInd(jobs, m, n)

gravaSched(maqs, m, 'I(4)')


n = 12
m = 3
dados = [[1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 6], [7, 7], [8, 8], [9, 9], [10, 10], [11, 11], [12, 12]]
I(4)
maq #  p  C 
 1  0  1  1
 1  5  6  7
 1  6  7 14
 1 11 12 26
 2  1  2  2
 2  4  5  7
 2  7  8 15
 2 10 11 26
 3  2  3  3
 3  3  4  7
 3  8  9 16
 3  9 10 26

Cmax: 26.0
Fbar: 12.5


Teste 5:

In [12]:
m,n,jobs=leInst(4)

maqs, Cmax, Fbar = calcInd(jobs, m, n)

gravaSched(maqs, m, 'I(5)')


n = 15
m = 5
dados = [[1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [6, 1], [7, 1], [8, 1], [9, 1], [10, 1], [11, 1], [12, 1], [13, 1], [14, 1], [15, 1]]
I(5)
maq #  p  C 
 1  0  1  1
 1  5  1  2
 1 10  1  3
 2  1  1  1
 2  6  1  2
 2 11  1  3
 3  2  1  1
 3  7  1  2
 3 12  1  3
 4  3  1  1
 4  8  1  2
 4 13  1  3
 5  4  1  1
 5  9  1  2
 5 14  1  3

Cmax:  3.0
Fbar:  2.0


Teste 6:

In [13]:
m,n,jobs=leInst(5)

maqs, Cmax, Fbar = calcInd(jobs, m, n)

gravaSched(maqs, m, 'I(6)')


n = 20
m = 5
dados = [[1, 13], [2, 48], [3, 55], [4, 76], [5, 63], [6, 61], [7, 90], [8, 19], [9, 48], [10, 94], [11, 55], [12, 32], [13, 37], [14, 14], [15, 14], [16, 15], [17, 24], [18, 17], [19, 70], [20, 2]]
I(6)
maq #  p  C 
 1  0 13 13
 1 17 17 30
 1  8 48 78
 1  9 94 172
 2 13 14 14
 2  7 19 33
 2  1 48 81
 2  6 90 171
 3 15 15 15
 3 16 24 39
 3 10 55 94
 3  3 76 170
 4 14 14 14
 4 11 32 46
 4  2 55 101
 4 18 70 171
 5 19  2  2
 5 12 37 39
 5  5 61 100
 5  4 63 163

Cmax: 172.0
Fbar: 77.3


Teste 7:

In [14]:
m,n,jobs=leInst(6)

maqs, Cmax, Fbar = calcInd(jobs, m, n)

gravaSched(maqs, m, 'I(7)')


n = 50
m = 5
dados = [[1, 27], [2, 59], [3, 91], [4, 25], [5, 70], [6, 51], [7, 61], [8, 98], [9, 98], [10, 77], [11, 48], [12, 81], [13, 53], [14, 7], [15, 95], [16, 56], [17, 1], [18, 48], [19, 92], [20, 42], [21, 61], [22, 19], [23, 93], [24, 9], [25, 60], [26, 59], [27, 71], [28, 69], [29, 11], [30, 36], [31, 85], [32, 82], [33, 87], [34, 36], [35, 13], [36, 9], [37, 61], [38, 85], [39, 82], [40, 99], [41, 17], [42, 24], [43, 32], [44, 77], [45, 45], [46, 18], [47, 72], [48, 44], [49, 21], [50, 48]]
I(7)
maq #  p  C 
 1 16  1  1
 1 40 17 18
 1 29 36 54
 1 33 36 90
 1 12 53 143
 1  1 59 202
 1 27 69 271
 1 38 82 353
 1 37 85 438
 1 39 99 537
 2 23  9  9
 2 45 18 27
 2  0 27 54
 2 47 44 98
 2 15 56 154
 2  6 61 215
 2 20 61 276
 2 11 81 357
 2 32 87 444
 2  7 98 542
 3 28 11 11
 3 34 13 24
 3 42 32 56
 3 19 42 98
 3  5 51 149
 3 25 59 208
 3  4 70 278
 3 31 82 360
 3 30 85 445
 3  8 98 543
 4 13  7  7
 4 48 21 28
 4  3 25 53
 4 44 45 98
 4 17 48 146
 4 36 61 207
 4 26 71 278
 4 43 7

Fecha o workbook de Dados

In [15]:
wb1.close()